## Importación de librerías

In [3]:
import pandas as pd
import os
import glob
import csv
import json

## Configuración de variables globales

In [15]:
with open('setup.json', 'r') as f:
    SETUP_JSON = json.load(f)
DATASETS_PATH = SETUP_JSON['datasets_path'] # Path to the datasets,
DATASETS_FOLDER = os.path.join(os.getcwd(), DATASETS_PATH) # Folder containing the datasets,
DATASETS = glob.glob(os.path.join(DATASETS_FOLDER, '*.csv')) # List of datasets
OUTPUT_CSV = SETUP_JSON['output_csv'] # Output CSV file
OUTPUT_PARQUET = SETUP_JSON['output_parquet'] # Output CSV file

## Obtener todas las características diferentes

In [3]:
datasets = [f"{DATASETS_PATH}\\02-14-2018.csv", f"{DATASETS_PATH}\\02-15-2018.csv", f"{DATASETS_PATH}\\02-16-2018.csv", f"{DATASETS_PATH}\\02-21-2018.csv"]

In [25]:
columns_lengths = []

# Obtener número de columnas por dataset
for dataset in DATASETS:
    columns_length = len(pd.read_csv(dataset, nrows=1).dtypes)
    columns_lengths.append(columns_length)

# Obtener grupo de características más largo
for dataset in DATASETS:
    df = pd.read_csv(dataset, nrows=1)
    columns_length = len(df.dtypes)
    if (columns_length == max(columns_lengths)):
        X_columns = df.dtypes.to_dict()
        X_columns.pop('Label')
print(len(X_columns))
for column, type in X_columns.items():
    print(column, type)

83
Flow ID object
Src IP object
Src Port int64
Dst IP object
Dst Port int64
Protocol int64
Timestamp object
Flow Duration int64
Tot Fwd Pkts int64
Tot Bwd Pkts int64
TotLen Fwd Pkts int64
TotLen Bwd Pkts int64
Fwd Pkt Len Max int64
Fwd Pkt Len Min int64
Fwd Pkt Len Mean float64
Fwd Pkt Len Std float64
Bwd Pkt Len Max int64
Bwd Pkt Len Min int64
Bwd Pkt Len Mean int64
Bwd Pkt Len Std float64
Flow Byts/s float64
Flow Pkts/s float64
Flow IAT Mean float64
Flow IAT Std float64
Flow IAT Max int64
Flow IAT Min int64
Fwd IAT Tot int64
Fwd IAT Mean float64
Fwd IAT Std float64
Fwd IAT Max int64
Fwd IAT Min int64
Bwd IAT Tot int64
Bwd IAT Mean float64
Bwd IAT Std float64
Bwd IAT Max int64
Bwd IAT Min int64
Fwd PSH Flags int64
Bwd PSH Flags int64
Fwd URG Flags int64
Bwd URG Flags int64
Fwd Header Len int64
Bwd Header Len int64
Fwd Pkts/s float64
Bwd Pkts/s float64
Pkt Len Min int64
Pkt Len Max int64
Pkt Len Mean float64
Pkt Len Std float64
Pkt Len Var float64
FIN Flag Cnt int64
SYN Flag Cnt int64


## Combinar datasets

Unimos todos los archivos CSV en un sólo dataframe y lo guardamos en formato CSV (para analizarlo con Tableau) y en formato Parquet (para su tratamiento con Python).

In [ ]:
df = pd.DataFrame()
for dataset in DATASETS:
    # Obtener las 10000 primeras filas del dataset
    current_df = pd.read_csv(dataset, nrows=10000, dtype=str) # Leer en formato string
    # Añadir columnas faltantes
    for column in X_columns.keys():
        if column not in current_df.columns:
            current_df[column] = "" # Rellenar columnas con cadenas vacías
    print(f"Dataset {dataset} procesado: {current_df.shape})")
    # Concatenar dataframe actual con el dataframe final
    df = pd.concat([df, current_df], ignore_index=True, sort=False)

Dataset c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-14-2018.csv procesado: (10000, 84))
Dataset c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-15-2018.csv procesado: (10000, 84))
Dataset c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-16-2018.csv procesado: (10000, 84))
Dataset c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-20-2018.csv procesado: (10000, 84))
Dataset c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-21-2018.csv procesado: (10000, 84))
Dataset c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-22-2018.csv procesado: (10000, 84))
Dataset c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-23-2018.csv procesado: (10000, 84))
Dataset c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-28-2018.csv procesado: (10000, 84))
Dataset c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-01-2018.csv procesado: (10000, 84))
D

## Exportación de archivos JSON

Exportar todas las columnas y sus tipos 

In [28]:
X_columns = {col: str(dtype) for col, dtype in X_columns.items()}

# Guardar en JSON
with open('X_columns.json', 'w') as f:
    json.dump(X_columns, f, indent=4)

In [29]:
# Obtener todas las etiquetas únicas del dataframe
labels_list = list(df['Label'].unique())
labels_list.remove('Label')

# Asignar índice a cada etiqueta
labels = {}
for i, label in enumerate(labels_list):
    labels[label] = i

# Guardar etiquetas en JSON
with open('labels.json', 'w') as f:
    json.dump(labels, f, indent=4)

## Exportación del dataframe en CSV y Parquet

In [30]:
# Guardar en CSV
df.to_csv(OUTPUT_CSV, index=False)
print(f"Archivo CSV guardado como {OUTPUT_CSV}")

# Guardar en Parquet
df.to_parquet(OUTPUT_PARQUET, index=False)
print(f"Archivo Parquet guardado como {OUTPUT_PARQUET}")

Archivo CSV guardado como merged_output.csv
Archivo Parquet guardado como merged_output.parquet


In [87]:

def find_datasets_with_same_header():
    headers = {}
    same_files = {}

    for file in DATASETS:
        with open(file, newline='', encoding='utf-8') as f:
            header = tuple(next(csv.reader(f), None))
            if header:
                if header in headers:
                    headers[header].append("datasets\\"+file)
                else:
                    headers[header] = ["datasets\\"+file]
    print("Archivos con headers distintos:", tuple(headers.values())[1])
    return tuple(headers.values())[0]

In [88]:
csv_files = find_datasets_with_same_header()
print("Archivos con el mismo encabezado:", csv_files)

Archivos con headers distintos: ['datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-20-2018.csv']
Archivos con el mismo encabezado: ['datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-14-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-15-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-16-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-21-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-22-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-23-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-28-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\03-01-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traff